In [1]:
# import pandas as pd
import MagmaPandas as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import meltInc.plotting as p
from functools import partial
from MagmaPandas.geochemistry.fO2 import fO2_QFM
from MagmaPandas.geochemistry.Kd_ol_melt import Kd_FeMg_vectorised
from MagmaPandas.geochemistry.Fe_redox import Fe_redox

p.layout()
config = mp.configuration()
config.print()

Kd Fe-Mg ol-melt.......................toplis
Melt Fe3+/Fe2+........................borisov
Melt thermometer...............putirka2008_14
Volatile solubility model......IaconoMarziano


In [2]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')

In [3]:
forsterite_host = 0.9
P_bar = 1e3

In [4]:
try:
    if len(forsterite_host) != melt.shape[0]:
        raise ValueError(
            "Host forsterite array length does not match the amount of melt inclusions"
        )
except TypeError:
    forsterite_host = pd.Series(forsterite_host, index=melt.index)
try:
    if len(P_bar) != melt.shape[0]:
        raise ValueError(
            "Pressure array length does not match the amount of melt inclusions"
        )
except TypeError:
    P_bar = pd.Series(P_bar, index=melt.index)



In [5]:
stepsize =  -0.001
converge = 0.002
temperature_converge = 0.2
QFM_logshift = 1
olivine_crystallised = pd.Series(0, index=melt.index)
decrease_factor = 10

In [6]:
inclusions = melt[melt.elements].copy()
inclusions = inclusions.normalise()
temperatures = inclusions.temperature(P_bar=P_bar)
fO2 = fO2_QFM(QFM_logshift, temperatures, P_bar)
Fe3Fe2_model = getattr(Fe_redox, config.Fe3Fe2_model)
Kd_model = getattr(Kd_FeMg_vectorised, config.Kd_model)

c:\data\dropbox\python\packages\magmapandas\src\MagmaPandas\geochemistry\fO2.py:161: UserWarning: O'Neill fO2: temperatures above 1420K present
  w.warn("O'Neill fO2: temperatures above 1420K present")


In [7]:
def calc_forsterite_EQ(
    mol_fractions,
    T_K=temperatures.copy(),
    P_bar=P_bar.copy(),
    oxygen_fugacity=fO2.copy(),
    Fo_initial=forsterite_host.copy(),
):
    # melt Fe3+/Fe2+
    Fe3Fe2 = Fe3Fe2_model(mol_fractions, T_K, oxygen_fugacity)
    # FeMg Kd
    Kd = Kd_model(mol_fractions, Fo_initial, T_K, P_bar, Fe3Fe2)
    Fe2_FeTotal = 1 / (1 + Fe3Fe2)
    Fe2Mg = mol_fractions["FeO"] * Fe2_FeTotal / mol_fractions["MgO"]
    # Equilibrium forsterite content
    return 1 / (1 + Kd * Fe2Mg)

In [8]:
moles = inclusions.moles
forsterite_EQ = calc_forsterite_EQ(moles)
disequilibrium =  ~ np.isclose(forsterite_EQ, forsterite_host, atol=converge)
stepsize = pd.Series(stepsize, index=inclusions.index)
stepsize.loc[forsterite_EQ > forsterite_host] = -stepsize
# Fe-Mg exchange vector
FeMg_exchange = pd.DataFrame(0, index=moles.index, columns=moles.columns)
FeMg_exchange.loc[:, ["FeO", "MgO"]] = np.stack((1, -1), axis=1)

In [9]:
moles_new = moles.loc[disequilibrium].copy()

moles_new = moles_new + FeMg_exchange.mul(stepsize, axis=0)
temperatures_new = moles_new.convert_moles_wtPercent.temperature(
    P_bar=P_bar
)
forsterite_EQ_new = calc_forsterite_EQ(moles_new)
olivine = pd.DataFrame(
    {
        "MgO": forsterite_EQ * 2,
        "FeO": (1 - forsterite_EQ_new) * 2,
        "SiO2": 1,
    },
    columns=moles_new.columns,
    index=moles_new.index,
).fillna(0.0)

olivine_stepsize = stepsize / 4
temperature_mismatch = ~ np.isclose(
    temperatures_new, temperatures, atol=temperature_converge
)
remove_olivine = moles_new.loc[temperature_mismatch].copy()

In [10]:
while sum(temperature_mismatch) > 1:
    remove_olivine = remove_olivine + olivine.mul(olivine_stepsize, axis=0)
    temperatures_new = remove_olivine.convert_moles_wtPercent.temperature(
        P_bar=P_bar
    )
    olivine_crystallised.loc[temperature_mismatch] += olivine_stepsize.loc[temperature_mismatch]
    temperature_mismatch = ~ np.isclose(
        temperatures_new, temperatures, atol=temperature_converge
    )

    T_overstepped = np.sign(temperatures - temperatures_new) != np.sign(
        stepsize
    )
    decrease_stepsize_T = np.logical_and(
        T_overstepped, temperature_mismatch
    )

    if sum(decrease_stepsize_T) > 0:
        remove_olivine.loc[decrease_stepsize_T] = (
            remove_olivine.loc[decrease_stepsize_T]
            - olivine.loc[decrease_stepsize_T].mul(olivine_stepsize.loc[decrease_stepsize_T], axis=0)
        )
        olivine_crystallised.loc[decrease_stepsize_T] = (
            olivine_crystallised.loc[decrease_stepsize_T]
            - olivine_stepsize.loc[decrease_stepsize_T]
        )
        olivine_stepsize.loc[decrease_stepsize_T] = (
            olivine_stepsize.loc[decrease_stepsize_T] / decrease_factor
        )


In [12]:
moles_new.loc[remove_olivine.index] = remove_olivine.copy()

forsterite_EQ_new = calc_forsterite_EQ(moles_new)

disequilibrium_new = ~ np.isclose(
    forsterite_EQ_new, forsterite_host.loc[disequilibrium], atol=converge
)
overstepped = np.sign(
    forsterite_EQ_new - forsterite_host.loc[disequilibrium]
) != np.sign(stepsize)
decrease_stepsize = np.logical_and(overstepped, disequilibrium_new)

In [18]:
stepsize

name
PI054-07-06   -0.001
PI054-07-05   -0.001
PI056-04-02   -0.001
PI116-04-01   -0.001
PI056-05-01   -0.001
PI089-03-01   -0.001
PI074-05-01   -0.001
PI052-02-01   -0.001
PI056-04-03   -0.001
PI085-04-02   -0.001
PI052-03-04   -0.001
PI085-04-05   -0.001
PI085-01-01   -0.001
PI085-01-06   -0.001
PI085-03-02   -0.001
PI052-03-02   -0.001
PI085-03-01   -0.001
PI085-01-05   -0.001
PI085-02-01   -0.001
PI052-05-01   -0.001
dtype: float64